<a href="https://colab.research.google.com/github/6058ah/BusinessCapstone/blob/master/%EB%89%B4%EC%8A%A4%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!pip install xlsxwriter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [226]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,내용요약본
- 내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> 엑셀로 저장
'''''''''''''''''''''

title_text=[]
contents_text=[]
dates_text = []
#result=[]

#엑셀로 저장하기 위한 변수
RESULT_PATH ='/content/drive'  #결과 저장할 경로

#날짜 범위 리스트로 만들기
def date_range(start, end):
    start = datetime.strptime(start, "%Y.%m.%d")
    end = datetime.strptime(end, "%Y.%m.%d")
    dates = [date.strftime("%Y.%m.%d") for date in pandas.date_range(start, periods=(end-start).days+1)]
    return dates

#def contents_cleansing(contents):
#    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '',str(contents)).strip()  #앞에 필요없는 부분 제거
#    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
#    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
#    contents_text.append(third_cleansing_contents)
#    #print(contents_text)

#크롤링 시작
def crawler(query,s_date,e_date):
  dates= date_range(s_date,e_date)
  page_num = [1,11,21,31,41]
  #maxpage_t =41
  for x in dates:
    print(x)
    for page_number in page_num:
      ds = x.replace(".","")
      de = ds
      url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0"+"&ds=" + x + "&de=" + x + "&nso=so%3Ar%2Cp%3Afrom" + ds + "to" + de + "%2Ca%3A&start=" + str(page_number)
      #url = "https://search.naver.com/search.naver?&where=news&query={0}&sm=tab_pge&sort={1}&photo=0&field=0&reporter_article=&pd=3&ds={2}&de={2}&docid=&nso=so:r,p:,a:all&mynews=1&cluster_rank=238&start={3}&refresh_start=0".format(query,sort,x,str(page_num))
      #print(url)
      response = requests.get(url)
      html = response.text

      #뷰티풀소프의 인자값 지정
      soup = BeautifulSoup(html, 'html.parser')

      #뉴스 날짜
      bbs = soup.find_all('span', 'info')
      for bb in bbs:
        t = bb.text
        dates_text.append(t)
      search = "면"
      for word in dates_text:
        if search in word: 
          dates_text.remove(word)


      #<a>태그에서 제목과 링크주소 (a 태그 중 class 명이 news_tit인 것)
      atags = soup.find_all('a', 'news_tit')
      for atag in atags:
        title = atag.get('title')
        title_text.append(title)     #제목
            
          #본문요약본 (a 태그 중 class 명이 api_txt_lines dsc_txt_wrap인 것)
      #contents_lists = soup.find_all('a','api_txt_lines dsc_txt_wrap')
      #for contents_list in contents_lists:
      #  contents_cleansing(contents_list) #본문요약 정제화


      #모든 리스트 딕셔너리형태로 저장
      #result= {"Date":dates_text,"title":title_text}
      df1 = pd.DataFrame(dates_text)
      df2 = pd.DataFrame(title_text) #df로 변환
      result = pd.concat([df1,df2],axis=1)
      print(result)

  outputFileName = '{0}~{1} {2} 관련 뉴스.xlsx'.format(s_date.replace(".",""), e_date.replace(".",""), query)
  writer = pd.ExcelWriter(outputFileName, engine='xlsxwriter')
  result.to_excel(writer, sheet_name= '{0}~{1} {2} 뉴스'.format(s_date.replace(".",""), e_date.replace(".",""), query))
  writer.save()

#메인함수
def main():
    info_main = input("="*50+"\n"+"입력 형식에 맞게 입력해주세요."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    query = input("검색어 입력: ") #네이버, 부동산...
    s_date = input("시작날짜 입력(예시:2019.01.01):")
    e_date = input("끝날짜 입력:")
    crawler(query,s_date,e_date)
  

In [227]:
main()

입력 형식에 맞게 입력해주세요.
 시작하시려면 Enter를 눌러주세요.
검색어 입력: 2차전지산업
시작날짜 입력(예시:2019.01.01):2021.05.01
끝날짜 입력:2021.05.03
2021.05.01
      0                                          0
0  2일 전            지난달 수출 512억 달러, 역대 4월 수출 1위 신기록
1  2일 전                     코로나 넘어선 4월 수출, 역대급 신기록
2  2일 전                 4월 수출 41.1%↑…10년 만에 최대폭 상승
3  2일 전            4월 수출, 역대 4월 최고 기록 경신…수출 성장세 진입
4  2일 전  ‘염블리’ 염승환 “주식투자 변수 5월 공매도 재개 아니라 6월 ‘이것’”
5  3일 전            "공매도 재개되면 바이오·배터리 거품 꺼진다" 고수 경고
      0                                          0
0  2일 전            지난달 수출 512억 달러, 역대 4월 수출 1위 신기록
1  2일 전                     코로나 넘어선 4월 수출, 역대급 신기록
2  2일 전                 4월 수출 41.1%↑…10년 만에 최대폭 상승
3  2일 전            4월 수출, 역대 4월 최고 기록 경신…수출 성장세 진입
4  2일 전  ‘염블리’ 염승환 “주식투자 변수 5월 공매도 재개 아니라 6월 ‘이것’”
5  3일 전            "공매도 재개되면 바이오·배터리 거품 꺼진다" 고수 경고
6  3일 전            "공매도 재개되면 바이오·배터리 거품 꺼진다" 고수 경고
      0                                          0
0  2일 전            지난달 수출 512억 달러, 역대 4월 수출 1위 신기록
1  2일 전        

#참고
##https://hansuho113.tistory.com/3?category=913503
##https://adsp-ggini.tistory.com/80